In [1]:
# system modules
import json
import pandas as pd
import sys, os
import ee

In [2]:
sys.dont_write_bytecode = True
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../../gee_toolbox'))

# user modules
import gee as gee_toolbox

from modules.SpectralIndexes import getEVI2
from modules.SpectralIndexes import getNDWI
from modules.Collection import getCollection
from modules.BandNames import getBandNames
from modules.DataType import setBandTypes
from modules.Mosaic import getMosaicAgriculture

# Variables configuration

In [3]:
# Set up version
VERSION = 7
MAPBIOMAS_COLLECTION_ID = 6.0
THEME = 'agriculture'
YEAR = 2016

ASSET_GRIDS = 'projects/mapbiomas-workspace/AUXILIAR/cartas'
ASSET_MOSAICS = 'projects/nexgenmap/MapBiomas2/SENTINEL/MOSAICS/agriculture'

COLLECTION_ID = 'COPERNICUS/S2'
SCALE = 10

# ancillary data
JSON_FILE = '../data/grid-names.json'
CSV_PERIODS = '../data/agriculture-periods.csv'

MAX_CLOUD_PROBABILITY = 40
MAX_CLOUD_COVER = 80

S2_CLOUD_PROBABILITY = 'COPERNICUS/S2_CLOUD_PROBABILITY'

ACCOUNT = 'mapbiomas2'

BUFFER_SIZE = 100

In [4]:
gee_toolbox.switch_user(ACCOUNT)
gee_toolbox.init()

gee.PREVIOUS_USER:
joao
gee.NEW_USER:
mapbiomas2
gee.init: USER_ACCOUNT 


# Ancillary data
## 1. Table of seasonality

In [5]:
df = pd.read_csv(CSV_PERIODS)

df

,grid_name,ent_t0,ent_t1,saf_t0,saf_t1
0,SA-23-X-C,Y1-09-01,Y2-02-15,Y2-02-01,Y2-06-30
1,SA-19-V-D,Y1-09-01,Y2-02-15,Y2-02-01,Y2-06-30
2,SA-21-V-D,Y1-09-01,Y2-02-15,Y2-02-01,Y2-06-30
3,SA-21-V-C,Y1-09-01,Y2-02-15,Y2-02-01,Y2-06-30
4,SA-21-X-C,Y1-09-01,Y2-02-15,Y2-02-01,Y2-06-30
...,...,...,...,...,...
553,SD-22-X-A,Y1-10-15,Y1-12-15,Y1-11-15,Y2-06-15
554,SD-24-V-D,Y1-10-15,Y1-12-15,Y1-11-15,Y2-06-15
555,SD-24-V-C,Y1-10-15,Y1-12-15,Y1-11-15,Y2-06-15
556,SB-24-Y-A,Y1-10-15,Y1-12-15,Y1-11-15,Y2-06-15


In [6]:
df = df.replace({'Y1':str(YEAR-1), 'Y2':str(YEAR)}, regex=True)

df

,grid_name,ent_t0,ent_t1,saf_t0,saf_t1
0,SA-23-X-C,2015-09-01,2016-02-15,2016-02-01,2016-06-30
1,SA-19-V-D,2015-09-01,2016-02-15,2016-02-01,2016-06-30
2,SA-21-V-D,2015-09-01,2016-02-15,2016-02-01,2016-06-30
3,SA-21-V-C,2015-09-01,2016-02-15,2016-02-01,2016-06-30
4,SA-21-X-C,2015-09-01,2016-02-15,2016-02-01,2016-06-30
...,...,...,...,...,...
553,SD-22-X-A,2015-10-15,2015-12-15,2015-11-15,2016-06-15
554,SD-24-V-D,2015-10-15,2015-12-15,2015-11-15,2016-06-15
555,SD-24-V-C,2015-10-15,2015-12-15,2015-11-15,2016-06-15
556,SB-24-Y-A,2015-10-15,2015-12-15,2015-11-15,2016-06-15


In [7]:
# test a data frame subset 
# df = df[df['grid_name'] == 'SD-23-Y-B']

df

,grid_name,ent_t0,ent_t1,saf_t0,saf_t1
0,SA-23-X-C,2015-09-01,2016-02-15,2016-02-01,2016-06-30
1,SA-19-V-D,2015-09-01,2016-02-15,2016-02-01,2016-06-30
2,SA-21-V-D,2015-09-01,2016-02-15,2016-02-01,2016-06-30
3,SA-21-V-C,2015-09-01,2016-02-15,2016-02-01,2016-06-30
4,SA-21-X-C,2015-09-01,2016-02-15,2016-02-01,2016-06-30
...,...,...,...,...,...
553,SD-22-X-A,2015-10-15,2015-12-15,2015-11-15,2016-06-15
554,SD-24-V-D,2015-10-15,2015-12-15,2015-11-15,2016-06-15
555,SD-24-V-C,2015-10-15,2015-12-15,2015-11-15,2016-06-15
556,SB-24-Y-A,2015-10-15,2015-12-15,2015-11-15,2016-06-15


In [8]:
ee.Initialize()

In [9]:
grids = ee.FeatureCollection(ASSET_GRIDS)

# Utilities functions

In [10]:
def applyCloudMask(collection, cloudProbability, maxCloudProbability=40):
    """
    """
    
    collectionWithCloudMask = collection.combine(cloudProbability)

    def cloudMasking(image):

        clouds = image.select('probability')

        notCloud = clouds.lte(maxCloudProbability)

        return image.updateMask(notCloud)

    collectionWithoutClouds = ee.ImageCollection(collectionWithCloudMask)\
        .map(cloudMasking)

    return collectionWithoutClouds

In [11]:
def multiplyBy10000(image):
    return image.multiply(10000)

def divideBy10000(image):
    return image.divide(10000)

# Iterating over table

In [12]:
for row in df.itertuples():

    # define a geometry
    grid = grids.filter(ee.Filter.stringContains(
        'grid_name', row.grid_name))
        
    grid = ee.Feature(grid.first()).geometry()\
        .buffer(BUFFER_SIZE).bounds()

    # returns a collection containing the specified parameters
    collectionEnt = getCollection(COLLECTION_ID,
                                collectionType='TOA',
                                dateStart=row.ent_t0,
                                dateEnd=row.ent_t1,
                                cloudCover=MAX_CLOUD_COVER,
                                geometry=grid
                                )
    
    collectionSaf = getCollection(COLLECTION_ID,
                                collectionType='TOA',
                                dateStart=row.saf_t0,
                                dateEnd=row.saf_t1,
                                cloudCover=MAX_CLOUD_COVER,
                                geometry=grid
                                )
    
    # cloud probability sentinel 2 collection
    cloudProbability = ee.ImageCollection(S2_CLOUD_PROBABILITY)

    cloudProbabilityEnt = cloudProbability\
        .filterDate(row.ent_t0, row.ent_t1)\
        .filterBounds(grid)
    
    cloudProbabilitySaf = cloudProbability\
        .filterDate(row.saf_t0, row.saf_t1)\
        .filterBounds(grid)
    
    # returns a pattern of band names
    bands = getBandNames('sentinel-2 (toa)')

    # Rename collection image bands
    collectionEnt = collectionEnt.select(
        bands['bandNames'],
        bands['newNames']
    )

    collectionSaf = collectionSaf.select(
        bands['bandNames'],
        bands['newNames']
    )
    
    # removes clouds
    collectionEnt = applyCloudMask(collectionEnt,
        cloudProbability=cloudProbabilityEnt,
        maxCloudProbability=MAX_CLOUD_PROBABILITY)
    
    collectionSaf = applyCloudMask(collectionSaf,
        cloudProbability=cloudProbabilitySaf,
        maxCloudProbability=MAX_CLOUD_PROBABILITY)
    
    # calculate Spectral indexes
    collectionEnt = collectionEnt\
        .map(divideBy10000)\
        .map(getEVI2)\
        .map(getNDWI)\
        .map(multiplyBy10000)
    
    collectionSaf = collectionSaf\
        .map(divideBy10000)\
        .map(getEVI2)\
        .map(getNDWI)\
        .map(multiplyBy10000)
    
    # dry season mosaic
    mosaicEnt = getMosaicAgriculture(collectionEnt,
        percentiles=[20,75],
        qualityBand='evi2')
    
    bandsEnt = mosaicEnt.bandNames().map(
        lambda band: ee.String(band).cat('_ent')
    )

    # rename dry seasom mosaic bands
    mosaicEnt = mosaicEnt.rename(bandsEnt)

    # wet season mosaic
    mosaicSaf = getMosaicAgriculture(collectionSaf,
        percentiles=[20,75],
        qualityBand='evi2')
    
    # rename wet season mosaic bands
    bandsSaf = mosaicSaf.bandNames().map(
        lambda band: ee.String(band).cat('_saf')
    )

    mosaicSaf = mosaicSaf.rename(bandsSaf)

    # merges the mosaics
    mosaic = mosaicSaf.addBands(mosaicEnt)
    
    # calculates annual evi2 cei
    exp = '100 * (b("evi2_qmo_saf") - b("evi2_p20_ent")) / (100 + b("evi2_qmo_saf") + 100 + b("evi2_p20_ent"))'

    cei = mosaic.expression(exp)\
        .rename(['evi2_cei_annual'])\
        .multiply(100)

    mosaic = mosaic.addBands(cei)

    # set data type for selected bands
    mosaic = setBandTypes(mosaic, mtype=THEME)

    # set properties
    mosaic = mosaic.set('year', YEAR)
    mosaic = mosaic.set('collection',   MAPBIOMAS_COLLECTION_ID)
    mosaic = mosaic.set('grid_name', row.grid_name)
    mosaic = mosaic.set('version', str(VERSION))
    mosaic = mosaic.set('theme', THEME)
    mosaic = mosaic.set('saf_t0', row.saf_t0)
    mosaic = mosaic.set('saf_t1', row.saf_t1)
    mosaic = mosaic.set('ent_t0', row.ent_t0)
    mosaic = mosaic.set('ent_t1', row.ent_t1)

    # creates the file name
    imageName = '{}-{}-{}-{}'.format(THEME.upper(), row.grid_name, YEAR, VERSION)

    print(imageName)

    task = ee.batch.Export.image.toAsset(
           image=mosaic,
           description=imageName,
           assetId=ASSET_MOSAICS + '/' + imageName,
           region=grid.coordinates().getInfo(),
           scale=SCALE,
           maxPixels=int(1e13)
        )
        
    task.start()

AGRICULTURE-SA-23-X-C-2016-7
AGRICULTURE-SA-19-V-D-2016-7
AGRICULTURE-SA-21-V-D-2016-7
AGRICULTURE-SA-21-V-C-2016-7
AGRICULTURE-SA-21-X-C-2016-7
AGRICULTURE-SA-21-X-D-2016-7
AGRICULTURE-SA-22-V-C-2016-7
AGRICULTURE-SA-22-V-D-2016-7
AGRICULTURE-SA-22-X-C-2016-7
AGRICULTURE-SA-22-X-D-2016-7
AGRICULTURE-SA-23-V-C-2016-7
AGRICULTURE-SA-23-V-D-2016-7
AGRICULTURE-SA-23-V-B-2016-7
AGRICULTURE-SB-18-Z-A-2016-7
AGRICULTURE-SA-19-X-D-2016-7
AGRICULTURE-SA-19-X-C-2016-7
AGRICULTURE-SA-20-V-C-2016-7
AGRICULTURE-SA-20-V-D-2016-7
AGRICULTURE-SA-20-X-C-2016-7
AGRICULTURE-SA-20-X-D-2016-7
AGRICULTURE-SB-22-X-B-2016-7
AGRICULTURE-SB-23-V-A-2016-7
AGRICULTURE-SB-23-V-B-2016-7
AGRICULTURE-SA-23-Z-A-2016-7
AGRICULTURE-SA-23-V-A-2016-7
AGRICULTURE-SA-19-V-B-2016-7
AGRICULTURE-SB-19-V-A-2016-7
AGRICULTURE-SB-18-X-B-2016-7
AGRICULTURE-SA-19-Y-B-2016-7
AGRICULTURE-SB-21-Z-B-2016-7
AGRICULTURE-SB-21-X-C-2016-7
AGRICULTURE-SB-21-X-D-2016-7
AGRICULTURE-SA-21-Z-C-2016-7
AGRICULTURE-SA-21-Z-D-2016-7
AGRICULTURE-SA

In [13]:
gee_toolbox.switch_user('joao')
gee_toolbox.init()

gee.PREVIOUS_USER:
mapbiomas2
gee.NEW_USER:
joao
gee.init: USER_ACCOUNT 
